# Zonal Statistics


# Introduction to code
1. calculate zonal statistics for (indirect nutrient load/stream)WatershedPourPoints_LH.shp
2. Calculate Zonal statistics for (direct nutrient flux/coastal wetland corresponding watershed)wetland_connected_avg_inundation_NAD1983_Watershed
3. Subtract the values of flux that have overlap 

# Zonal Statistics for Coastal Watersheds

This notebook calculates zonal statistics/Nutrient flux for the coastal watersheds in the Great Lakes region.


## Objectives:
1. **Integrate Coastal Watershed Data**:
   - Utilize the delineated coastal watersheds as zones for statistical analysis.

2. **Compute Spatial Metrics**:
   - Perform calculations, such as mean, sum, and area-weighted averages, for variables like:
     - Land use types.
     - Vegetation cover.
     - Hydrological properties.

3. **Export Results for Further Analysis**:
   - Aggregate and save the results in a structured format to facilitate downstream analysis and visualization.

## Workflow:
1. **Data Loading**:
   - Import relevant spatial datasets, including raster and vector files.
   - Integrate delineated coastal watershed data from previous analyses.

2. **Zonal Statistics Calculation**:
   - Apply zonal statistics tools to compute desired metrics over each watershed zone.

3. **Result Validation and Export**:
   - Validate calculated statistics.
   - Export the results as tables or geospatial files for further use.

---

This notebook is a critical step in quantifying spatial characteristics within the Great Lakes coastal watersheds, supporting environmental management and decision-making.


# Required Imports

In [1]:
### Reqiured Imports###
import pandas, os
import arcpy
import pandas as pd
import numpy as np

# Inputs

In [3]:

# Define the base paths
base_path = r'D:\Users\abolmaal\Arcgis\NASAOceanProject'
nutrient_flux_path = r'D:\Users\abolmaal\Arcgis\NASAOceanProject\Luwen_Nutrient'
coastal_watersheds_path = r'D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Erase_Lake'
#TotalLOad = r'D:\Users\abolmaal\Arcgis\NASAOceanProject\ZonalStats\CoastalWatershed_total_N_P_avg._directandIndirect.csv'
# Define the input paths
inDB = os.path.join(base_path, 'GIS_layer', 'Streamwatershed')


# Nutrient flux rasters
inBYRaster_TN = os.path.join(nutrient_flux_path, 'TN_Annual_delTotal_header_kgcellday.tif')
inBYRaster_TP = os.path.join(nutrient_flux_path, 'TP_Annual_delTotal_header_kgcellday.tif')

# Define the input feature classes
inPourpoint = 'WatershedPourPoints_NAD1983'

inWatershed = os.path.join(inDB, 'PointWatershed_LH_NAD1983_NASA_Invasive.shp')

# Define coastal watersheds
inCW = os.path.join(coastal_watersheds_path, 'Erase_drainage')
inCoastalWatershed_low = os.path.join(coastal_watersheds_path,'CoastalWatershed_low_erase_lakedrain_LakeHuron.shp')
inCoastalWatershed_high = os.path.join(coastal_watersheds_path,'CoastalWatershed_high_erase_lakedrain_LakeHuron.shp')
inCoastalWatershed_avg = os.path.join(coastal_watersheds_path, 'CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp')
inCoastalWatershed_surge = os.path.join(coastal_watersheds_path,'CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp')

# Output coastal feature
outTable = 'direct_coastal_N_P_wetland_loads'
outFeature = 'wetland_connected_avg_inundation_Albers'

# Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.workspace = inDB

# Temporary files
tempZonal = 'temp_zonal_stats'


# path to Lake Huron shapefile
Lake_Huron = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\hydro_p_LakeHuron\hydro_p_LakeHuron_84.shp"


In [ ]:
inCoastalWatershed_avg=r'D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp'

In [4]:
print("File exists:", os.path.exists(inCoastalWatershed_avg))
print("Full path:", inCoastalWatershed_avg)

File exists: True
Full path: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Erase_Lake\CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp


# Outputs

In [3]:
outBYRaster_TN =  os.path.join(nutrient_flux_path, 'TN_Annual_kgcellday_header_kgcellday_converted.tif')
outByRaster_TP =  os.path.join(nutrient_flux_path, 'TP_Annual_delTotal_header_kgcellday_converted.tif')
# Fields to calculate / Direct delivery to Watersheds
StreamDirectTN = 'StreamDirectTN_kgday'
StreamDirectTN_converted = 'StreamDirectTN_grm2yr'
StreamDirectTP = 'StreamDirectTP_kgday'
StreamDirectTP_converted = 'StreamDirectTP_grm2yr'

# Direct delivery to Coastal Watersheds
CoastalDirectTN_avg = 'CoastalWatershedDirectTN_avg_kgday'
CoastalDirectTN_avg_convert = 'CoastalWatershedDirectTN_avg_grm2yr'
CoastalDirectTP_avg = 'CoastalWatershedDirectTP_avg_kgday'
CoastalDirectTP_avg_convert = 'CoastalWatershedDirectTP_avg_grm2yr'


CoastalDirectTN_low = 'CoastalWatershedDirectTN_low_kgday'
CoastalDirectTN_low_convert = 'CoastalWatershedDirectTN_low_grm2yr'
CoastalDirectTP_low = 'CoastalWatershedDirectTP_low_kgday'
CoastalDirectTP_low_convert = 'CoastalWatershedDirectTP_low_grm2yr'


CoastalDirectTN_high = 'CoastalWatershedDirectTN_high_kgday'
CoastalDirectTN_high_convert = 'CoastalWatershedDirectTN_high_grm2yr'
CoastalDirectTP_high_convert = 'CoastalWatershedDirectTP_high_grm2yr'
CoastalDirectTP_high = 'CoastalWatershedDirectTP_high_kgday'

CoastalDirectTN_surge = 'CoastalWatershedDirectTN_surge_kgday'
CoastalDirectTN_surge_convert = 'CoastalWatershedDirectTN_surge_grm2yr'
CoastalDirectTP_surge = 'CoastalWatershedDirectTP_surge_kgday'
CoastalDirectTP_surge_convert = 'CoastalWatershedDirectTP_surge_grm2yr'

# ##############################################
# fieldCoastalLimTN = 'CoastLoad_TN_lim_kg2'
# fieldCoastalLimTP = 'CoastLoad_TP_lim_kg2'

# # Ratios
# fieldDirectRatio = 'WetLoad_TN_TP_ratio'
# fieldCoastalRatio = 'CoastLoad_TN_TP_ratio'
# fieldCoastalDirectRatioTN = 'Coast_Direct_ratio_TN'
# fieldCoastalDirectRatioTP = 'Coast_Direct_ratio_TP'

# out path
outDb = os.path.join(base_path, 'ZonalStats')

outFig = os.path.join(base_path, 'figures')


# Parameters

In [4]:
# define the watershed number, this number is the same as the FID number and it is used to number the watersheds and we will use this for particle tracking
watershed_num = 'StWater_id'
# define the coastal wetland number which is same as FID number+1
Coastal_num = 'CW_Id'
Area = 'CW_Area'

# Main Functions

In [ ]:
# before doing any thing conevt the b=values of TN and TP to kgcellday
import os
import rasterio
import numpy as np

# Define paths


# Read the raster
with rasterio.open(inBYRaster_TN) as src:
    TN = src.read(1).astype('float32')
    profile = src.profile
    nodata = src.nodata

# Handle NoData values
if nodata is not None:
    TN = np.where(TN == nodata, np.nan, TN)

# Print min and max BEFORE conversion
print("Before conversion (kg/cell/day):")
print(f"Min: {np.nanmin(TN):.6f}, Max: {np.nanmax(TN):.6f}")

# Constants
cell_size_km = 0.12
cell_area_km2 = cell_size_km ** 2  # 0.0144 km²
days_per_year = 365

# Convert to kg/yr/km²
converted_data = (TN * days_per_year) / cell_area_km2

# Print min and max AFTER conversion
print("After conversion (kg/yr/km²):")
print(f"Min: {np.nanmin(converted_data):.6f}, Max: {np.nanmax(converted_data):.6f}")

# Restore NoData values
if nodata is not None:
    converted_data = np.where(np.isnan(converted_data), nodata, converted_data)

# Update metadata
profile.update(dtype=rasterio.float32, compress='lzw')

# Save the output raster
with rasterio.open(outBYRaster_TN, 'w', **profile) as dst:
    dst.write(converted_data.astype(rasterio.float32), 1)

print("Conversion complete. Saved as:", outBYRaster_TN)


In [ ]:
# before doing any thing conevt the b=values of TN and TP to kgcellday
import rasterio
# Open the input raster
with rasterio.open(inBYRaster_TN) as src:
    TP = src.read(1)  # Read the first band
    profile = src.profile  # Get metadata
    nodata_value = src.nodata  # Get NoData value from metadata

# Debugging: Print initial statistics
print("Original TN stats:")
print(f"Min: {np.nanmin(TP)}, Max: {np.nanmax(TP)}, Mean: {np.nanmean(TP)}")




## Direct NUtrint Load to Stream Watersheds

####  Bring in Streamwatershed into a pandas dataframe and get the specific columns


In [ ]:
# Bring in Streamwatershed into a pandas dataframe and get the specific columns

dfWatershed = pandas.DataFrame(arcpy.da.TableToNumPyArray(os.path.join(inDB,inWatershed), [watershed_num, 'gridcode','Shape_Area']))

In [ ]:

dfWatershed


### Estimate Zonal statistics for StreamWatreshed
Bring in the nutrient flux rasters into a pandas dataframe and estimate the total nutrient load in the Streamwatershed

In [ ]:
# 1- calculate the zonal statistics with arcpy for inWatershed using inBYRaster
inZonalTN_watershed = arcpy.sa.ZonalStatisticsAsTable(inWatershed, watershed_num, inBYRaster_TN, 'SUM' ,'DATA')

# # bring in inWatershed and inZonalTN_watershed as pandas dataframes
dfZonalTN_watershed = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTN_watershed,
                                                                  [watershed_num, 'AREA','SUM']))

#rename the SUM field to fieldDirectTN
dfZonalTN_watershed.rename(columns={'SUM': StreamDirectTN}, inplace=True)
dfZonalTN_watershed


In [ ]:
dfZonalTN_watershed.describe()

In [ ]:
inZonalTP_watershed = arcpy.sa.ZonalStatisticsAsTable(inWatershed,watershed_num, inBYRaster_TP, 'SUM' ,'DATA')
dfZonalTP_watershed = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTP_watershed, 
                                                                   [watershed_num, 'AREA','SUM']))
# rename the SUM field to fieldDirectTP
dfZonalTP_watershed.rename(columns={'SUM': StreamDirectTP}, inplace=True)

dfZonalTP_watershed.describe()

###  Merge Streamwatershed for TN and TP

In [ ]:
# merge dfWateshed with dfZonalTN_watershed where dfWateshed['Group_id'] = dfZonalTN_watershed['Group_id']
dfWatershed = dfWatershed.merge(dfZonalTP_watershed, left_on= watershed_num, right_on= watershed_num, how='inner')

In [ ]:
# merge dfWateshed with dfZonalTN_watershed where dfWateshed['Group_id'] = dfZonalTN_watershed['Group_id']
dfWatershed = dfWatershed.merge(dfZonalTN_watershed, left_on=watershed_num, right_on=watershed_num, how='inner')
dfWatershed

In [ ]:
# remove the unnecessary columns from dfWatershed
dfWatershed.drop(columns=['gridcode', 'AREA_x','AREA_y','gridcode'], inplace=True)
dfWatershed

In [ ]:
dfWatershed.columns

### convert the unit to grm2year

In [ ]:
# Define the conversion function
def convert_kgday_to_grm2yr(df, kgday_col, area_col, new_col):
    df[new_col] = (df[kgday_col] * 1000 * 365) / df[area_col]

# Define the area column name
area_col = "Shape_Area"

# Define conversion mappings for each dataframe
conversion_sets = {
    "dfWatershed": [
        ("StreamDirectTN_kgday", "StreamDirectTN_grm2yr"),
        ("StreamDirectTP_kgday_y", "StreamDirectTP_grm2yr")]
    
}
 # Perform the conversion for each DataFrame
for df_name, conversions in conversion_sets.items():
    df = globals()[df_name]  # Get the actual DataFrame object by name
    for kgday_col, grm2yr_col in conversions:
        if kgday_col in df.columns:
            convert_kgday_to_grm2yr(df, kgday_col, area_col, grm2yr_col)
        else:
            print(f"⚠️ Column '{kgday_col}' not found in {df_name}, skipping.")

In [ ]:
dfWatershed

### Save stream watershed Nutrient load

In [ ]:
# delete the extra columns then save them and rename them that make sense
# convert dfWatershed to a pandas dataframe
dfWatershed_pd = pd.DataFrame(dfWatershed)
# save dfWatershed_pd to a shape file in outDb with name outTable + '.shp'
dfWatershed_pd.to_csv(os.path.join(outDb, outTable + '.shp'))

In [ ]:
# Save the dfWateshed to a csv file in the inDB directory with the name of Watresheds_total_N_P.csv
dfWatershed.to_csv(os.path.join(outDb, 'StreamWatresheds_total_N_P.csv'), index=False)

## 2- Zonal Statistics for coastal Wetlands watershed for Different Innundation

Calculate the zonal statistics with arcpy for inCoastawateshed using inBYRaster_TN, and inBYRaster_TP


In [5]:
#bring in the coastalwatreshed

dfCoastalWatershed_avg = pandas.DataFrame(arcpy.da.TableToNumPyArray(os.path.join(inCW,inCoastalWatershed_avg),
                                                                  [Coastal_num, 'gridcode','CW_Area']))

dfCoastalWatershed_low = pandas.DataFrame(arcpy.da.TableToNumPyArray(os.path.join(inCW,inCoastalWatershed_low),
                                                                  [Coastal_num, 'gridcode','CW_Area']))

dfCoastalWatershed_high = pandas.DataFrame(arcpy.da.TableToNumPyArray(os.path.join(inCW,inCoastalWatershed_high),
                                                                  [Coastal_num, 'gridcode','CW_Area']))

dfCoastalWatershed_surge = pandas.DataFrame(arcpy.da.TableToNumPyArray(os.path.join(inCW,inCoastalWatershed_surge),
                                                                  [Coastal_num, 'gridcode','CW_Area']))

#### Average Innundation

In [7]:
# 1- Calculate the zonal statistics with arcpy for inCoastalWatershed average inundation using inBYRaster_TN and inBYRaster_TP
inZonalTN_coastal_avg = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_avg, Coastal_num, inBYRaster_TN, 'SUM' ,'DATA')


In [8]:
# # bring in inCoastalWatershed and inZonalTN_coastal as pandas dataframes
dfZonalTN_coastal_avg = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTN_coastal_avg,
                                                                  [Coastal_num, 'AREA','SUM']))
# add a field to dfZonal_TN_coastal_avg called fieldCoastalTN_avg which is equal to SUM column divided by Shape_Area
dfZonalTN_coastal_avg.rename(columns={'SUM': CoastalDirectTN_avg}, inplace=True)

In [12]:
# print min and max and average of this column CoastalDirectTN_avg 
print("CoastalDirectTN_avg stats:")
print(f"Min: {dfZonalTN_coastal_avg[CoastalDirectTN_avg].min():.6f}, "
      f"Max: {dfZonalTN_coastal_avg[CoastalDirectTN_avg].max():.6f}, "
      f"Mean: {dfZonalTN_coastal_avg[CoastalDirectTN_avg].mean():.6f}")

CoastalDirectTN_avg stats:
Min: 0.000158, Max: 41.108646, Mean: 0.376504


In [10]:
inZonalTP_coastal_avg = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_avg, Coastal_num, inBYRaster_TP, 'SUM' ,'DATA')

dfZonalTP_coastal_avg = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTP_coastal_avg,
                                                                  [Coastal_num, 'AREA','SUM']))
# add a field to dfZonal_TP_coastal_avg called fieldCoastalTP_avg which is equal to SUM column divided by Shape_Area
dfZonalTP_coastal_avg.rename(columns={'SUM': CoastalDirectTP_avg}, inplace=True)


In [13]:
# print min and max and average of the dfZonalTP_coastal_avg['CoastalDirectTP_avg']
print("CoastalDirectTP_avg stats:")
print(f"Min: {dfZonalTP_coastal_avg[CoastalDirectTP_avg].min():.6f}, "
      f"Max: {dfZonalTP_coastal_avg[CoastalDirectTP_avg].max():.6f}, "
      f"Mean: {dfZonalTP_coastal_avg[CoastalDirectTP_avg].mean():.6f}")

CoastalDirectTP_avg stats:
Min: 0.000029, Max: 70.952182, Mean: 0.048538


In [14]:
# merge dfZonalTN_coastal_avg and dfZonalTP_coastal_avg with dfCoastalWatershed_avg
dfCoastalWatershed_avg = dfCoastalWatershed_avg.merge(dfZonalTN_coastal_avg, left_on=Coastal_num, right_on=Coastal_num, how='inner')
dfCoastalWatershed_avg = dfCoastalWatershed_avg.merge(dfZonalTP_coastal_avg, left_on=Coastal_num, right_on=Coastal_num, how='inner')

In [15]:
dfCoastalWatershed_avg

,CW_Id,gridcode,CW_Area,AREA_x,CoastalWatershedDirectTN_avg_kgday,AREA_y,CoastalWatershedDirectTP_avg_kgday
0,2,2,240307,43200.0,0.060420,43200.0,0.001673
1,14,24,10000,14400.0,0.028460,14400.0,0.000442
2,32,53,150387,158400.0,0.157051,158400.0,0.006932
3,33,58,42978,28800.0,0.016162,28800.0,0.000394
4,34,67,148190,43200.0,0.058000,43200.0,0.001385
...,...,...,...,...,...,...,...
2088,10160,17922,392500,388800.0,0.610330,388800.0,0.026406
2089,10161,18008,47500,43200.0,0.032592,43200.0,0.009584
2090,10162,17997,15000,14400.0,0.003813,14400.0,0.000402
2091,10165,18015,77500,72000.0,0.511640,72000.0,0.002618


In [16]:
dfCoastalWatershed_avg.drop(columns=['gridcode', 'AREA_x','AREA_y'], inplace=True)


In [17]:
dfCoastalWatershed_avg.columns

Index(['CW_Id', 'CW_Area', 'CoastalWatershedDirectTN_avg_kgday',
       'CoastalWatershedDirectTP_avg_kgday'],
      dtype='object')

In [18]:
CoastalDirectTN_avg_convert

'CoastalWatershedDirectTN_avg_grm2yr'

In [19]:
dfCoastalWatershed_avg[CoastalDirectTN_avg_convert] = (dfCoastalWatershed_avg[CoastalDirectTN_avg] * 1000 * 365) / dfCoastalWatershed_avg['CW_Area']
dfCoastalWatershed_avg[CoastalDirectTP_avg_convert] = (dfCoastalWatershed_avg[CoastalDirectTP_avg] * 1000 * 365) / dfCoastalWatershed_avg['CW_Area']
# print the min and max and average of the dfCoastalWatershed_avg[CoastalDirectTN_avg_convert] and dfCoastalWatershed_avg[CoastalDirectTP_avg_convert]
print("CoastalDirectTN_avg_convert stats:")
print(f"Min: {dfCoastalWatershed_avg[CoastalDirectTN_avg_convert].min():.6f}, "
      f"Max: {dfCoastalWatershed_avg[CoastalDirectTN_avg_convert].max():.6f}, "
      f"Mean: {dfCoastalWatershed_avg[CoastalDirectTN_avg_convert].mean():.6f}")

print("CoastalDirectTP_avg_convert stats:")
print(f"Min: {dfCoastalWatershed_avg[CoastalDirectTP_avg_convert].min():.6f}, "
      f"Max: {dfCoastalWatershed_avg[CoastalDirectTP_avg_convert].max():.6f}, "
      f"Mean: {dfCoastalWatershed_avg[CoastalDirectTP_avg_convert].mean():.6f}")
    


CoastalDirectTN_avg_convert stats:
Min: 0.000465, Max: 34.645799, Mean: 1.221888
CoastalDirectTP_avg_convert stats:
Min: 0.000009, Max: 8.695754, Mean: 0.060400


In [20]:
outDb

'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\ZonalStats'

In [21]:
# save the dfCoastalWatershed_avg to a csv file in the outDb directory with the name of CoastalWatersheds_avg_total_N_P_new6225.csv
dfCoastalWatershed_avg.to_csv(os.path.join(outDb, 'CoastalWatersheds_avg_total_N_P_new6225.csv'), index=False)


### High inundation 


In [ ]:
# 1- Calculate the zonal statistics with arcpy for inCoastalWatershed High inundation using inBYRaster_TN and inBYRaster_TP
inZonalTN_coastal_high = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_high, Coastal_num, inBYRaster_TN, 'SUM' ,'DATA')
inZonalTP_coastal_high = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_high, Coastal_num, inBYRaster_TP, 'SUM' ,'DATA')

# # bring in inCoastalWatershed and inZonalTN_coastal as pandas dataframes
dfZonalTN_coastal_high = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTN_coastal_high,
                                                                  [Coastal_num, 'AREA','SUM']))

dfZonalTP_coastal_high = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTP_coastal_high,
                                                                  [Coastal_num, 'AREA','SUM']))

In [ ]:
# add a field to dfZonal_TN_coastal_avg called fieldCoastalTN_high which is equal to SUM column divided by Shape_Area
dfZonalTN_coastal_high.rename(columns={'SUM': CoastalDirectTN_high}, inplace=True)


# add a field to dfZonal_TP_coastal_avg called fieldCoastalTP_avg which is equal to SUM column divided by Shape_Area
dfZonalTP_coastal_high.rename(columns={'SUM': CoastalDirectTP_high}, inplace=True)


In [ ]:
# merge dfZonalTN_coastal_avg and dfZonalTP_coastal_avg with dfCoastalWatershed_high
dfCoastalWatershed_high = dfCoastalWatershed_high.merge(dfZonalTN_coastal_high, left_on=Coastal_num, right_on=Coastal_num, how='inner')
dfCoastalWatershed_high = dfCoastalWatershed_high.merge(dfZonalTP_coastal_high, left_on=Coastal_num, right_on=Coastal_num, how='inner')

In [ ]:
dfCoastalWatershed_high.drop(columns=['gridcode', 'AREA_x','AREA_y'], inplace=True)

### Low inundation


In [ ]:
# 1- Calculate the zonal statistics with arcpy for inCoastalWatershed low inundation using inBYRaster_TN and inBYRaster_TP
inZonalTN_coastal_low = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_low, Coastal_num, inBYRaster_TN, 'SUM' ,'DATA')
inZonalTP_coastal_low = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_low, Coastal_num, inBYRaster_TP, 'SUM' ,'DATA')

# # bring in inCoastalWatershed and inZonalTN_coastal as pandas dataframes
dfZonalTN_coastal_low = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTN_coastal_low,
                                                                  [Coastal_num, 'AREA','SUM']))

dfZonalTP_coastal_low = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTP_coastal_low,
                                                                  [Coastal_num, 'AREA','SUM']))

In [ ]:
# add a field to dfZonal_TN_coastal_avg called fieldCoastalTN_avg which is equal to SUM column divided by Shape_Area
dfZonalTN_coastal_low.rename(columns={'SUM': CoastalDirectTN_low}, inplace=True)


# add a field to dfZonal_TP_coastal_avg called fieldCoastalTP_avg which is equal to SUM column divided by Shape_Area
dfZonalTP_coastal_low.rename(columns={'SUM': CoastalDirectTP_low}, inplace=True)


In [ ]:
# merge dfZonalTN_coastal_avg and dfZonalTP_coastal_avg with dfCoastalWatershed_avg
dfCoastalWatershed_low = dfCoastalWatershed_low.merge(dfZonalTN_coastal_low, left_on=Coastal_num, right_on=Coastal_num, how='inner')
dfCoastalWatershed_low = dfCoastalWatershed_low.merge(dfZonalTP_coastal_low, left_on=Coastal_num, right_on=Coastal_num, how='inner')

In [ ]:
dfCoastalWatershed_low.drop(columns=['gridcode', 'AREA_x','AREA_y'], inplace=True)

### Surge inundation


In [ ]:
# 1- Calculate the zonal statistics with arcpy for inCoastalWatershed average inundation using inBYRaster_TN and inBYRaster_TP
inZonalTN_coastal_surge = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_surge, Coastal_num, inBYRaster_TN, 'SUM' ,'DATA')
inZonalTP_coastal_surge = arcpy.sa.ZonalStatisticsAsTable(inCoastalWatershed_surge, Coastal_num, inBYRaster_TP, 'SUM' ,'DATA')

# # bring in inCoastalWatershed and inZonalTN_coastal as pandas dataframes
dfZonalTN_coastal_surge = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTN_coastal_surge,
                                                                  [Coastal_num, 'AREA','SUM']))

dfZonalTP_coastal_surge = pandas.DataFrame(arcpy.da.TableToNumPyArray(inZonalTP_coastal_surge,
                                                                  [Coastal_num, 'AREA','SUM']))

In [ ]:
# add a field to dfZonal_TN_coastal_avg called fieldCoastalTN_avg which is eual to SUM column divided by Shape_Area
dfZonalTN_coastal_surge.rename(columns={'SUM': CoastalDirectTN_surge}, inplace=True)


# add a field to dfZonal_TP_coastal_avg called fieldCoastalTP_avg which is eual to SUM column divided by Shape_Area
dfZonalTP_coastal_surge.rename(columns={'SUM': CoastalDirectTP_surge}, inplace=True)


In [ ]:
# merge dfZonalTN_coastal_avg and dfZonalTP_coastal_avg with dfCoastalWatershed_surge
dfCoastalWatershed_surge = dfCoastalWatershed_surge.merge(dfZonalTN_coastal_surge, left_on=Coastal_num, right_on=Coastal_num, how='inner')
dfCoastalWatershed_surge = dfCoastalWatershed_surge.merge(dfZonalTP_coastal_surge, left_on=Coastal_num, right_on=Coastal_num, how='inner')

In [ ]:
dfCoastalWatershed_surge.drop(columns=['gridcode','AREA_x','AREA_y'], inplace=True)

### conver the values of kgda to grm2peryear 

In [ ]:
# Define the conversion function
def convert_kgday_to_grm2yr(df, kgday_col, area_col, new_col):
    df[new_col] = (df[kgday_col] * 1000 * 365) / df[area_col]

# Define the area column name
area_col = "CW_Area"

# Define conversion mappings for each dataframe
conversion_sets = {
    "dfCoastalWatershed_avg": [
        ("CoastalWatershedDirectTN_avg_kgday", "CoastalWatershedDirectTN_avg_grm2yr"),
        ("CoastalWatershedDirectTP_avg_kgday", "CoastalWatershedDirectTP_avg_grm2yr"),
    ],
    "dfCoastalWatershed_low": [
        ("CoastalWatershedDirectTN_low_kgday", "CoastalWatershedDirectTN_low_grm2yr"),
        ("CoastalWatershedDirectTP_low_kgday", "CoastalWatershedDirectTP_low_grm2yr"),
    ],
    "dfCoastalWatershed_high": [
        ("CoastalWatershedDirectTN_high_kgday", "CoastalWatershedDirectTN_high_grm2yr"),
        ("CoastalWatershedDirectTP_high_kgday", "CoastalWatershedDirectTP_high_grm2yr"),
    ],
    "dfCoastalWatershed_surge": [
        ("CoastalWatershedDirectTN_surge_kgday", "CoastalWatershedDirectTN_surge_grm2yr"),
        ("CoastalWatershedDirectTP_surge_kgday", "CoastalWatershedDirectTP_surge_grm2yr"),
    ]
}
   


In [ ]:
# Perform the conversion for each DataFrame
for df_name, conversions in conversion_sets.items():
    df = globals()[df_name]  # Get the actual DataFrame object by name
    for kgday_col, grm2yr_col in conversions:
        if kgday_col in df.columns:
            convert_kgday_to_grm2yr(df, kgday_col, area_col, grm2yr_col)
        else:
            print(f"⚠️ Column '{kgday_col}' not found in {df_name}, skipping.")


In [ ]:
# show the dfCoastalWatershed_avg min and max values
print("Coastal Watershed Average TN Min:", dfCoastalWatershed_avg['CoastalWatershedDirectTN_avg_grm2yr'].min())
print("Coastal Watershed Average TN Max:", dfCoastalWatershed_avg['CoastalWatershedDirectTN_avg_grm2yr'].max())
print ("Coastal Watershed Average TN median:", dfCoastalWatershed_avg['CoastalWatershedDirectTN_avg_grm2yr'].median())
print("Coastal Watershed Average TP Min:", dfCoastalWatershed_avg['CoastalWatershedDirectTP_avg_grm2yr'].min())
print("Coastal Watershed Average TP Max:", dfCoastalWatershed_avg['CoastalWatershedDirectTP_avg_grm2yr'].max())
print ("Coastal Watershed Average TP median:", dfCoastalWatershed_avg['CoastalWatershedDirectTP_avg_grm2yr'].median())

In [ ]:
dfCoastalWatershed_avg.describe()


## 2-5 Save the dfCoastalWatershed as csv

In [ ]:
# save the dfZonal_coastal to a csv file in the inDB directory with the name of CoastalWatershed_total_N_P.csv
dfCoastalWatershed_avg.to_csv(os.path.join(outDb, 'CoastalWatershed_total_N_P_avg_last.csv'), index=False)
dfCoastalWatershed_low.to_csv(os.path.join(outDb, 'CoastalWatershed_total_N_P_low.csv'), index=False)
dfCoastalWatershed_high.to_csv(os.path.join(outDb, 'CoastalWatershed_total_N_P_high.csv'), index=False)
dfCoastalWatershed_surge.to_csv(os.path.join(outDb, 'CoastalWatershed_total_N_P_surge.csv'), index=False)


In [ ]:
outDb

# Plots

## Plot Direct Nutrient Load to StreamWatershed

In [ ]:
#dfZonalTP_watershed
# add a line to calculate the ratio of TN/TP
plt.figure(figsize=(10, 8))
dfZonalTN_watershed[fieldDirectTN].plot(color='red', label='TN')
dfZonalTP_watershed[fieldDirectTP].plot(color='blue', label='TP')
# plot log scale
plt.yscale('log')
plt.xlabel('Watershed ID')
plt.ylabel('Nutrient Load (kg/day)')
plt.title('Direct Nutrient Load in Stream Watersheds')
plt.grid()
plt.legend()
plt.savefig(os.path.join(outFig, 'Direct_Nutrient_Load_Stream_Watersheds.png'))
plt.show()

In [ ]:
# Calcaulte the ratio of TN/TP for the direct delivery to watersheds and add it to dfWatershed then plot it
dfWatershed[fieldDirectRatio] = dfWatershed[fieldDirectTN]/dfWatershed[fieldDirectTP]
dfWatershed[fieldDirectRatio].plot(color='green', label='TN/TP', linestyle='--')
plt.xlabel('Watershed ID')
plt.ylabel('TN/TP')
plt.title('Direct Nutrient Load Ratio in Stream Watersheds')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Calculate mean and standard error for fieldDirectTN and fieldDirectTP and fieldDirectRatio 
# plot a column graph with error bars for the mean and standard error for each field
mean_directTN = dfWatershed[fieldDirectTN].mean()
std_directTN = dfWatershed[fieldDirectTN].std()

mean_directTP = dfWatershed[fieldDirectTP].mean()
std_directTP = dfWatershed[fieldDirectTP].std()



fig, ax = plt.subplots()
ax.bar(['TN', 'TP'], [mean_directTN, mean_directTP,], yerr=[std_directTN, std_directTP],color = ['red', 'blue'])
plt.ylabel('Nutrient Load (kgcellday/area)')

plt.title('Mean Direct Nutrient Load in Stream Watersheds per Area per Day')
plt.show()


## Plot Direct Nutrient Load to Coastal Watersheds

In [ ]:
dfCoastalWatershed_high[fieldCoastalTN_high].plot(color='yellow', label='TN_coastal_High')
dfCoastalWatershed_avg[fieldCoastalTN_avg].plot(color='blue', label='TN_coastal_Average',zorder=2)
dfCoastalWatershed_low[fieldCoastalTN_low].plot(color='green', label='TN_coastal_Low', zorder=1)
dfCoastalWatershed_surge[fieldCoastalTN_surge].plot(color='pink', label='TN_coastal_Surge', zorder=3)
plt.xlabel('Coastal Watershed ID')
plt.ylabel('Nitrogen Load (kg/day)')
plt.yscale('log')
plt.style.use('ggplot')
plt.title('Direct Nitrogen Load in Coastal Watersheds')
plt.grid()
plt.legend()
filename = 'Nitrogen_Load_in_Coastal_Watersheds.png'
outputname = os.path.join(outFig, filename)
plt.savefig(outputname)

## boxplot

In [ ]:
# Apply the '_mpl-gallery' style
plt.style.use('_mpl-gallery')

# Prepare the data for the box plot
data_N = [
    dfCoastalWatershed_high[fieldCoastalTN_high],
    dfCoastalWatershed_avg[fieldCoastalTN_avg],
    dfCoastalWatershed_low[fieldCoastalTN_low],
    dfCoastalWatershed_surge[fieldCoastalTN_surge]
]
# Calculate the mean and standard deviation for each category
means = [d.mean() for d in data_N]
stds = [d.std() for d in data_N]

# Plot the box plot with means and standard deviations
fig, ax = plt.subplots(figsize=(8, 8))
box = ax.boxplot(data_N, labels=['High', 'Average', 'Low', 'Surge'], patch_artist=True, showmeans=False, meanline=False,
                 medianprops={"color": "white", "linewidth": 0.5},
                 whiskerprops={"color": "C0", "linewidth": 1.5},
                 capprops={"color": "C0", "linewidth": 1.5},
                 meanprops={"color": "red", "linewidth": 1.5},
                 showfliers=False)

# Set different colors for each box
colors = ['red', 'blue', 'green', 'black']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    
# Set labels and title
plt.xlabel('Coastal Watershed Category')
plt.ylabel('Nitrogen Load (kg/day)')
plt.title('Direct Nitrogen Load in Coastal Watersheds')

filename = 'Nitrogen_Load_in_Coastal_Watersheds_boxplot.png'
output_path = os.path.join(outFig, filename)
plt.savefig(output_path, dpi= 300)
plt.show()

In [ ]:
# Apply the '_mpl-gallery' style
plt.style.use('_mpl-gallery')

# Prepare the data (ensure these are numeric lists)
data_P = [
    dfCoastalWatershed_high[fieldCoastalTP_high],
    dfCoastalWatershed_avg[fieldCoastalTP_avg],
    dfCoastalWatershed_low[fieldCoastalTP_low],
    dfCoastalWatershed_surge[fieldCoastalTP_surge]
]

# Calculate the mean and standard deviation for each category
means = [np.mean(d) for d in data_P]
stds = [np.std(d) for d in data_P]

# Plot the box plot
fig, ax = plt.subplots(figsize=(8, 8))
box = ax.boxplot(data_P, labels=['High', 'Average', 'Low', 'Surge'], patch_artist=True, showmeans=True, meanline=True,
                 medianprops={"color": "white", "linewidth": 0.5},
                 whiskerprops={"color": "C0", "linewidth": 1.5},
                 capprops={"color": "C0", "linewidth": 1.5},
                 meanprops={"color": "red", "linewidth": 1.5},
                 showfliers=False)

# Set different colors for each box
colors = ['red', 'blue', 'green', 'black']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Set labels and title
plt.xlabel('Coastal Watershed Category')
plt.ylabel('Phosphorus Load (kgcellday)')
plt.title('Phosphorus Load in Coastal Watersheds')

filename = 'Phosphorus_Load_in_Coastal_Watersheds_boxplot.png'
output_path = os.path.join(outFig, filename)
plt.savefig(output_path, dpi= 300)
